In [1]:
from sklearn.ensemble import IsolationForest
from sklearn import svm
import pandas as pd
import numpy as np
import sys
import csv

params = {}
params['n_estimators'] = 100
params['max_samples'] ='auto'
params['contamination'] = 0.1
params['max_features'] = 1.0

params['nu'] = 0.001
params['gamma']='auto'
params['kernel'] ='poly'

params['path'] = 'data/train/B01.csv'
params['opath'] ='dataclean/B01.csv'
try:
    with open(params['path'],'r') as f:
        reader = csv.reader(f)
        head_row=next(reader)
        data_attribute = []
    for item in head_row:
        data_attribute.append(item)
    print(data_attribute)
except Exception as e:
    print(e)

['DE_time', 'FE_time', 'BA_time', 'RPM']


In [2]:
tn = pd.read_csv(params['path']) 
tn.dropna(inplace=True)
train = np.array(tn)
train_x = np.array(train)
print(train_x)
# clf = IsolationForest(n_estimators=params['n_estimators'], 
#                       max_samples=params['max_samples'], 
#                       contamination=params['contamination'], 
#                       max_features=params['max_features'], 
#                       bootstrap=False, n_jobs=1, random_state=None, 
#                       verbose=0).fit(train_x)
clf = svm.OneClassSVM(nu=params['nu'],
              kernel=params['kernel'],
              gamma=params['gamma']).fit(train_x)
pred = clf.predict(train_x)
print (pred.size)

[[ 1.91511018e-01 -3.69818182e-02 -8.20842730e-03  1.79600000e+03]
 [ 1.50577365e-01  4.49945455e-02 -1.55316320e-02  1.79600000e+03]
 [-3.68727745e-02 -8.62909091e-03  3.82255193e-03  1.79600000e+03]
 ...
 [ 2.42353214e-01 -6.98545455e-03  7.06970920e-02  1.79600000e+03]
 [ 3.24870259e-03  1.06836364e-02  7.12604154e-02  1.79600000e+03]
 [-6.22126547e-02 -1.01700000e-01 -2.03198813e-02  1.79600000e+03]]
14708


In [7]:
df = pd.DataFrame(pd.read_csv(params['path']))[0:pred.size]
df['pred']=pred
df2 = df[-df.pred.isin([-1])]
df2 = df2.drop(['pred'],axis=1)
data_out = df2.iloc[:,:].values
print(int(df2.size/4))
csvfile2 = open(params['opath'],'w',newline='')
writer = csv.writer(csvfile2)
writer.writerow(data_attribute)   #存属性
m=len(data_out)
for i in range(m):
    writer.writerow(data_out[i])

14708


In [28]:
import pandas as pd
import numpy as np
import os
from scipy import stats,signal,fftpack
import math
from pywt import wavedec
import traceback
import sys
columns_list=['time_mean','time_std','time_max','time_min','time_rms','time_ptp','time_median','time_iqr','time_pr','time_skew','time_kurtosis','time_var','time_amp',                'time_smr','time_wavefactor','time_peakfactor','time_pulse','time_margin','freq_mean','freq_std','freq_max','freq_min','freq_rms','freq_median',               'freq_iqr','freq_pr','freq_f2','freq_f3','freq_f4','freq_f5','freq_f6','freq_f7','freq_f8','ener_cA5','ener_cD1','ener_cD2','ener_cD3','ener_cD4',        'ener_cD5','ratio_cA5','ratio_cD1','ratio_cD2','ratio_cD3','ratio_cD4','ratio_cD5']
columns_list1 = [a + 'DE' for a in columns_list]
columns_list2 = [a + 'FE' for a in columns_list]
columns_list3 = [a + 'BA' for a in columns_list]
columns_list_final = columns_list1 + columns_list2+ columns_list3
params['data_path'] = 'data/train/B01.csv'
params['len_piece']=100 #窗口长度
params['opath'] ='datafeature/B01.csv'
files_path = params['data_path']
windowlen=params['len_piece']


def feature_get(filepath,windowlen):
        df_data = pd.DataFrame(pd.read_csv(filepath))
        dfs = df_data.loc[:,['DE_time','FE_time','BA_time']]
        feature_list = []
        print(len(dfs))
        for i in range (0,len(dfs),windowlen):
            df=dfs[i:i+params['len_piece']]
#             print(df)
            for i in df.columns:
                #----------  time-domain feature,18
                #依次为均值，标准差，最大值，最小值，均方根，峰峰值，中位数，四分位差，百分位差，偏度，峰度，方差，整流平均值，方根幅值，波形因子，峰值因子，脉冲值，裕度
                df_line = df[i]
                print(df_line)
                time_mean = df_line.mean()
                time_std = df_line.std()
                time_max = df_line.max()
                time_min = df_line.min()
                time_rms = np.sqrt(np.square(df_line).mean())
                time_ptp = time_max-time_min 
                time_median = np.median(df_line)
                time_iqr = np.percentile(df_line,75)-np.percentile(df_line,25)
                time_pr = np.percentile(df_line,90)-np.percentile(df_line,10)
                time_skew = stats.skew(df_line)
                time_kurtosis = stats.kurtosis(df_line)
                time_var = np.var(df_line)
                time_amp = np.abs(df_line).mean()
                time_smr = np.square(np.sqrt(np.abs(df_line)).mean())
                #下面四个特征需要注意分母为0或接近0问题，可能会发生报错
                time_wavefactor = time_rms/time_amp
                time_peakfactor = time_max/time_rms
                time_pulse = time_max/time_amp
                time_margin = time_max/time_smr
                #----------  freq-domain feature,15
                #采样频率25600Hz
                df_fftline = fftpack.fft(df[i])
                freq_fftline = fftpack.fftfreq(len(df[i]),1/25600)
                df_fftline = abs(df_fftline[freq_fftline>=0])
                freq_fftline = freq_fftline[freq_fftline>=0]
                #基本特征,依次为均值，标准差，最大值，最小值，均方根，中位数，四分位差，百分位差
                freq_mean = df_fftline.mean()
                freq_std = df_fftline.std()
                freq_max = df_fftline.max()
                freq_min = df_fftline.min()
                freq_rms = np.sqrt(np.square(df_fftline).mean())
                freq_median = np.median(df_fftline)
                freq_iqr = np.percentile(df_fftline,75)-np.percentile(df_fftline,25)
                freq_pr = np.percentile(df_fftline,90)-np.percentile(df_fftline,10)
                #f2 f3 f4反映频谱集中程度
                freq_f2 = np.square((df_fftline-freq_mean)).sum()/(len(df_fftline)-1)
                freq_f3 = pow((df_fftline-freq_mean),3).sum()/(len(df_fftline)*pow(freq_f2,1.5))
                freq_f4 = pow((df_fftline-freq_mean),4).sum()/(len(df_fftline)*pow(freq_f2,2))
                #f5 f6 f7 f8反映主频带位置
                freq_f5 = np.multiply(freq_fftline,df_fftline).sum()/df_fftline.sum()
                freq_f6 = np.sqrt(np.multiply(np.square(freq_fftline),df_fftline).sum())/df_fftline.sum()
                freq_f7 = np.sqrt(np.multiply(pow(freq_fftline,4),df_fftline).sum())/np.multiply(np.square(freq_fftline),df_fftline).sum()
                freq_f8 = np.multiply(np.square(freq_fftline),df_fftline).sum()/np.sqrt(np.multiply(pow(freq_fftline,4),df_fftline).sum()*df_fftline.sum())
                #----------  timefreq-domain feature,12
                #5级小波变换，最后输出6个能量特征和其归一化能量特征
                cA5, cD5, cD4, cD3, cD2, cD1 = wavedec(df[i], 'db10', level=5)
                ener_cA5 = np.square(cA5).sum()
                ener_cD5 = np.square(cD5).sum()
                ener_cD4 = np.square(cD4).sum()
                ener_cD3 = np.square(cD3).sum()
                ener_cD2 = np.square(cD2).sum()
                ener_cD1 = np.square(cD1).sum()
                ener = ener_cA5 + ener_cD1 + ener_cD2 + ener_cD3 + ener_cD4 + ener_cD5
                ratio_cA5 = ener_cA5/ener
                ratio_cD5 = ener_cD5/ener
                ratio_cD4 = ener_cD4/ener
                ratio_cD3 = ener_cD3/ener
                ratio_cD2 = ener_cD2/ener
                ratio_cD1 = ener_cD1/ener
                feature_list.append([time_mean,time_std,time_max,time_min,time_rms,time_ptp,time_median,time_iqr,time_pr,time_skew,time_kurtosis,time_var,time_amp,
                                 time_smr,time_wavefactor,time_peakfactor,time_pulse,time_margin,freq_mean,freq_std,freq_max,freq_min,freq_rms,freq_median,
                                 freq_iqr,freq_pr,freq_f2,freq_f3,freq_f4,freq_f5,freq_f6,freq_f7,freq_f8,ener_cA5,ener_cD1,ener_cD2,ener_cD3,ener_cD4,ener_cD5,
                                 ratio_cA5,ratio_cD1,ratio_cD2,ratio_cD3,ratio_cD4,ratio_cD5])

        return feature_list

fea = feature_get(files_path,windowlen)
print(fea)
wrtocsv = pd.DataFrame(fea)
wrtocsv.to_csv(params['opath'],index=False,header=False)
# features.append(fea)

14708
0     0.191511
1     0.150577
2    -0.036873
3    -0.076182
4     0.029563
        ...   
95   -0.176729
96   -0.064649
97    0.244790
98    0.030863
99   -0.168608
Name: DE_time, Length: 100, dtype: float64
0    -0.036982
1     0.044995
2    -0.008629
3     0.104782
4     0.128409
        ...   
95    0.028558
96    0.053418
97   -0.019724
98   -0.056500
99    0.151831
Name: FE_time, Length: 100, dtype: float64
0    -0.008208
1    -0.015532
2     0.003823
3     0.021125
4    -0.009375
        ...   
95    0.082004
96    0.096087
97   -0.002736
98   -0.013640
99   -0.004788
Name: BA_time, Length: 100, dtype: float64
100    0.204343
101    0.267856
102   -0.152364
103   -0.074720
104    0.215714
         ...   
195   -0.216201
196   -0.077806
197    0.144892
198    0.008934
199   -0.075857
Name: DE_time, Length: 100, dtype: float64
100    0.166418
101    0.106220
102    0.288869
103   -0.047049
104    0.047460
         ...   
195    0.134778
196   -0.072525
197   -0.028558
198    

1700   -0.021766
1701   -0.034924
1702    0.023391
1703    0.060101
1704    0.103634
          ...   
1795    0.026152
1796   -0.046944
1797   -0.043370
1798   -0.019655
1799    0.022903
Name: DE_time, Length: 100, dtype: float64
1700    0.077456
1701   -0.057527
1702    0.059376
1703    0.123067
1704    0.022600
          ...   
1795    0.032256
1796    0.129436
1797   -0.003698
1798   -0.054856
1799    0.046433
Name: FE_time, Length: 100, dtype: float64
1700    0.074600
1701    0.033236
1702    0.057620
1703   -0.022332
1704   -0.035530
          ...   
1795    0.016377
1796    0.007323
1797    0.029695
1798    0.070697
1799    0.038547
Name: BA_time, Length: 100, dtype: float64
1800   -0.012832
1801   -0.016731
1802    0.038335
1803    0.081542
1804    0.061238
          ...   
1895    0.271429
1896    0.005523
1897   -0.153339
1898    0.096162
1899    0.187613
Name: DE_time, Length: 100, dtype: float64
1800   -0.140736
1801    0.022395
1802    0.027736
1803    0.038831
1804    0.13

3300    0.067116
3301    0.047722
3302    0.148999
3303    0.138859
3304    0.050055
          ...   
3395   -0.005030
3396   -0.001127
3397   -0.046273
3398    0.003420
3399    0.029414
Name: BA_time, Length: 100, dtype: float64
3400    0.186638
3401   -0.016081
3402   -0.140831
3403    0.032487
3404    0.085766
          ...   
3495   -0.002112
3496    0.185988
3497    0.034761
3498    0.009909
3499    0.212303
Name: DE_time, Length: 100, dtype: float64
3400   -0.069649
3401   -0.007602
3402    0.068211
3403   -0.097180
3404   -0.028969
          ...   
3495    0.035749
3496   -0.015409
3497    0.025682
3498   -0.007807
3499   -0.005342
Name: FE_time, Length: 100, dtype: float64
3400    0.045026
3401    0.105744
3402    0.070053
3403    0.037863
3404    0.049210
          ...   
3495   -0.004386
3496    0.023136
3497    0.054079
3498   -0.040801
3499   -0.022292
Name: BA_time, Length: 100, dtype: float64
3500    0.075857
3501   -0.133684
3502    0.029238
3503    0.217176
3504   -0.01

5000    0.067389
5001    0.073142
5002    0.044173
5003    0.088551
5004    0.061020
          ...   
5095    0.140120
5096    0.025271
5097    0.197647
5098    0.048693
5099   -0.060198
Name: FE_time, Length: 100, dtype: float64
5000   -0.015210
5001   -0.041485
5002   -0.021406
5003   -0.020078
5004    0.022090
          ...   
5095    0.008369
5096   -0.009657
5097    0.076491
5098    0.057821
5099    0.018067
Name: BA_time, Length: 100, dtype: float64
5100    0.007634
5101   -0.106557
5102    0.166496
5103    0.179816
5104   -0.092426
          ...   
5195    0.159349
5196    0.121664
5197    0.003898
5198   -0.090151
5199    0.040934
Name: DE_time, Length: 100, dtype: float64
5100    0.191689
5101    0.032667
5102   -0.052391
5103    0.111356
5104    0.113205
          ...   
5195    0.005958
5196    0.095331
5197   -0.032462
5198    0.016436
5199    0.108480
Name: FE_time, Length: 100, dtype: float64
5100    0.067317
5101    0.008450
5102   -0.015733
5103    0.047400
5104    0.00

6600    0.164384
6601    0.022903
6602   -0.067735
6603   -0.006173
6604    0.029401
          ...   
6695   -0.216688
6696    0.145379
6697    0.182740
6698   -0.149115
6699   -0.155613
Name: DE_time, Length: 100, dtype: float64
6600    0.014382
6601    0.008629
6602    0.110329
6603    0.103344
6604    0.066156
          ...   
6695    0.107042
6696   -0.059582
6697   -0.094920
6698    0.134367
6699    0.058144
Name: FE_time, Length: 100, dtype: float64
6600    0.036093
6601    0.001167
6602    0.040036
6603    0.021165
6604   -0.000201
          ...   
6695    0.027482
6696   -0.055407
6697   -0.055206
6698   -0.023458
6699   -0.063495
Name: BA_time, Length: 100, dtype: float64
6700    0.073908
6701    0.054091
6702   -0.087390
6703   -0.117116
6704   -0.021604
          ...   
6795    0.304728
6796   -0.170394
6797   -0.148303
6798    0.465539
6799    0.293683
Name: DE_time, Length: 100, dtype: float64
6700   -0.009451
6701    0.069033
6702   -0.081565
6703    0.075196
6704    0.08

8200    0.009746
8201   -0.140994
8202    0.084466
8203    0.265094
8204   -0.043045
          ...   
8295    0.138557
8296    0.081380
8297    0.021929
8298   -0.070334
8299   -0.005035
Name: DE_time, Length: 100, dtype: float64
8200    0.072731
8201   -0.096564
8202   -0.035338
8203    0.174431
8204    0.001644
          ...   
8295    0.016642
8296    0.063485
8297   -0.044378
8298    0.107247
8299    0.172376
Name: FE_time, Length: 100, dtype: float64
8200    0.064098
8201    0.016940
8202    0.010542
8203    0.038950
8204    0.021044
          ...   
8295    0.081923
8296    0.058022
8297    0.011226
8298    0.045750
8299    0.008812
Name: BA_time, Length: 100, dtype: float64
8300    0.098598
8301   -0.017218
8302   -0.095025
8303    0.028913
8304    0.032162
          ...   
8395   -0.024528
8396    0.172993
8397    0.122476
8398   -0.102821
8399   -0.082517
Name: DE_time, Length: 100, dtype: float64
8300    0.010478
8301    0.099851
8302    0.050953
8303    0.033489
8304    0.19

9800    0.071087
9801    0.038625
9802   -0.102316
9803    0.023011
9804   -0.117931
          ...   
9895   -0.026298
9896    0.031640
9897    0.033078
9898    0.102933
9899    0.046844
Name: FE_time, Length: 100, dtype: float64
9800    0.033558
9801    0.070858
9802    0.041485
9803    0.020762
9804   -0.021688
          ...   
9895    0.016457
9896    0.057982
9897    0.012192
9898    0.025873
9899    0.061362
Name: BA_time, Length: 100, dtype: float64
9900    0.275490
9901   -0.003736
9902   -0.312688
9903   -0.017705
9904    0.223023
          ...   
9995   -0.087715
9996    0.165359
9997    0.244627
9998    0.109481
9999   -0.064812
Name: DE_time, Length: 100, dtype: float64
9900    0.001849
9901    0.012122
9902    0.138271
9903    0.042735
9904   -0.074991
          ...   
9995    0.127382
9996   -0.064513
9997    0.057733
9998    0.019518
9999   -0.003287
Name: FE_time, Length: 100, dtype: float64
9900    0.006398
9901    0.034443
9902    0.048647
9903   -0.035248
9904   -0.03

11300    0.009255
11301   -0.011146
11302    0.034765
11303    0.007042
11304    0.029253
           ...   
11395    0.064420
11396    0.014244
11397   -0.014807
11398    0.000966
11399   -0.033679
Name: BA_time, Length: 100, dtype: float64
11400   -0.109806
11401   -0.007634
11402    0.107857
11403   -0.007310
11404   -0.183064
           ...   
11495    0.034111
11496    0.058802
11497    0.026802
11498    0.069685
11499    0.075532
Name: DE_time, Length: 100, dtype: float64
11400    0.049309
11401    0.130053
11402    0.157995
11403    0.075196
11404    0.105604
           ...   
11495   -0.062047
11496    0.052391
11497    0.130669
11498   -0.084647
11499    0.050542
Name: FE_time, Length: 100, dtype: float64
11400   -0.040398
11401   -0.017262
11402   -0.035892
11403    0.027844
11404    0.093029
           ...   
11495   -0.014687
11496    0.029494
11497    0.025993
11498    0.000402
11499    0.039231
Name: BA_time, Length: 100, dtype: float64
11500    0.076994
11501    0.201582


12900    0.054160
12901    0.009858
12902    0.032351
12903    0.003259
12904   -0.041404
           ...   
12995    0.042491
12996    0.067398
12997    0.041284
12998    0.073272
12999    0.075767
Name: BA_time, Length: 100, dtype: float64
13000   -0.041421
13001   -0.105908
13002    0.117116
13003    0.024690
13004   -0.106557
           ...   
13095    0.075370
13096    0.363855
13097   -0.036873
13098   -0.283287
13099    0.070822
Name: DE_time, Length: 100, dtype: float64
13000   -0.031435
13001   -0.008629
13002    0.202167
13003   -0.072320
13004    0.057116
           ...   
13095    0.064102
13096   -0.102316
13097    0.115260
13098   -0.010684
13099   -0.029380
Name: FE_time, Length: 100, dtype: float64
13000    0.035489
13001    0.054924
13002    0.024786
13003   -0.015411
13004    0.000885
           ...   
13095   -0.031104
13096   -0.020038
13097    0.035489
13098   -0.001368
13099    0.028528
Name: BA_time, Length: 100, dtype: float64
13100    0.190212
13101   -0.052791


14500    0.028206
14501   -0.028247
14502   -0.040318
14503    0.006197
14504   -0.040921
           ...   
14595   -0.001851
14596    0.020441
14597    0.001690
14598    0.018670
14599    0.035047
Name: BA_time, Length: 100, dtype: float64
14600    0.217176
14601    0.030050
14602   -0.134496
14603    0.005035
14604    0.039959
           ...   
14695   -0.022091
14696   -0.104933
14697    0.098761
14698    0.131410
14699   -0.145379
Name: DE_time, Length: 100, dtype: float64
14600    0.134778
14601   -0.141558
14602    0.067184
14603    0.024860
14604   -0.173609
           ...   
14695    0.228055
14696    0.027736
14697   -0.015820
14698    0.012944
14699    0.047665
Name: FE_time, Length: 100, dtype: float64
14600   -0.011548
14601    0.017986
14602    0.066271
14603    0.014284
14604    0.015532
           ...   
14695    0.101841
14696    0.039956
14697    0.026999
14698   -0.026315
14699   -0.034966
Name: BA_time, Length: 100, dtype: float64
14700   -0.096974
14701    0.185501
